In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense,LSTM,Activation
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from sklearn.cross_validation import train_test_split


Using TensorFlow backend.
/home/riktim/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
pat=pd.read_csv('string.csv',names = ["pattern"])
pat.head(2)
#print(len(df["pattern"][0]))


,pattern
0,'CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHCCCCC...
1,'CCCHHHHHHHHHHHHHHHHHHCHHHHHHHHHHHHHHHHHCHHHHH...


In [4]:
class_file_name=pd.read_csv('id_label.csv',names = ["class", "file_name"])
class_file_name.head(2)

,class,file_name
0,1,'1ab3A.secstr'
1,1,'1abvA.secstr'


In [5]:
pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 1 columns):
pattern    551 non-null object
dtypes: object(1)
memory usage: 4.4+ KB


In [6]:
pat.describe()

,pattern
count,551
unique,551
top,'CCEEEEEECCCHHHHHHHHHHHHHHHHHCCCCEEEEEECCCHHHH...
freq,1


In [7]:
data=pd.DataFrame()
data['pattern']=pat['pattern']
data['class_no']=class_file_name['class']
data['pattern']=[s[1:-1] for s in data.pattern]

In [8]:
data.tail()

,pattern,class_no
546,CCCCEEEEEEEECCCCCCCCCCEEEEEECCCCCEEECCCHHHHHHH...,4
547,CCCCCCCCCCCCCCCCEEEEEECHHHCCCEECCCCCCCCCCCCCEE...,4
548,CEEEEEECCHHHHHHHCCCCCEEEEECCCCCCCCEEEEEEECCCCC...,4
549,CCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHCCEEEEEEEC...,4
550,CCCHHHHHHHHHCCCCCCCCCCCHHHHHHHHHHCCCCCCCCCEEEE...,4


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 2 columns):
pattern     551 non-null object
class_no    551 non-null int64
dtypes: int64(1), object(1)
memory usage: 8.7+ KB


In [10]:
X = data.pattern
Y = data.class_no
#le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)
X=[s.lower() for s in X]

In [11]:
print(X[0])

ccccccccccccccccccccccchhhhhhhhhhhhhhhhhcccccccccccccccccchhhccchhhchhhhcchhhcchhhcccccc


In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [13]:
type(X_train[0])

str

In [14]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(X_train)

In [18]:
X_train[300]

'cchhhhhhhhhhhhhhhccccccchhhhhhhhhhhhhhhccchhhhhhhhhhhhhhhccccchhhhhhhhhhhhhhhhhhhhhhhhhhhhhhccccccchhhhhhhhhhhhhhhhhhhhhhhhhcccccchhhhhhhhhhhhhhhhhhhhhhhhhhhhccchhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhcccchhhhhhhhhhhhhhhhhhhhhhhchhhhcccccccceeccccceeecccccccccccccccchhhhhhccccccccceeeeeeeeeeeecccccccceeeeeeeeeeeecccccccceeccceecccceeeeeeeccceeeeeeeeeeeeeeecceeeeeeeeeeeeeeecccceeeeeeeecccccceeeeehhhcccccccccchhhhcceeeeeeeeeecccccceeeeeeeeeccccccceecccceeeeehhhcceeccceeeeccccccccceeeeccceeeeeeeceecccccceeeeeeeeeeccceeeeeeecceeeeeeeeccccccccccchhhceeeecccceecccceeeeeeeccccccceeeeeeeeeeec'

In [19]:
#alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
alphabet="abcdefghijklmnopqrstuvwxyz"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1


In [21]:
#char_dict

In [22]:
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

In [23]:
# Convert string to index
train_sequences = tk.texts_to_sequences(X_train)
test_texts = tk.texts_to_sequences(X_test)

AttributeError: 'list' object has no attribute 'len'

In [59]:

# Padding
train_data = pad_sequences(train_sequences, maxlen=900, padding='post')
test_data = pad_sequences(test_texts, maxlen=900, padding='post')


In [60]:
# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')


In [61]:
train_data.shape
#print(test_data.shape)

(468, 900)

In [62]:
train_classes = Y_train.values
train_class_list = [x - 1 for x in train_classes]

test_classes = Y_test.values
test_class_list = [x - 1 for x in test_classes]

In [63]:
print(train_classes.shape)

(468,)


In [64]:
from keras.utils import to_categorical

train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [65]:
input_size = 900
max_len=input_size
vocab_size = len(tk.word_index)
#embedding_size = 69
embedding_size=27
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 4
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()


Load
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 900)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 900, 27)           756       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 894, 256)          48640     
_________________________________________________________________
activation_7 (Activation)    (None, 894, 256)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 298, 256)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 292, 256)          459008    
_________________________________________________________________
activation_8 (Activation)    (None, 292, 256)          0         
_____

In [66]:
def RNN():
    inputs=Input(name='inputs',shape=[max_len])
    layer=Embedding(vocab_size,50,input_length=max_len)(inputs)
    layer=LSTM(64)(layer)
    layer=Dense(256,name='FC1')(layer)
    layer=Activation('relu')(layer)
    layer=Dropout(0.5)(layer)
    layer=Dense(4,name='out_layer')(layer)
    layer=Activation('softmax')(layer)
    model1=Model(inputs=inputs,outputs=layer)
    return model1

In [67]:
model1=RNN()
model1.summary()
model1.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 900)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 900, 50)           1350      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_13 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 4)                 1028      
__________

In [68]:
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

In [69]:
model1.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
verbose=2)

Train on 468 samples, validate on 83 samples
Epoch 1/10
 - 15s - loss: 1.3853 - acc: 0.2692 - val_loss: 1.3860 - val_acc: 0.1928
Epoch 2/10
 - 13s - loss: 1.3846 - acc: 0.2756 - val_loss: 1.3868 - val_acc: 0.1928
Epoch 3/10
 - 13s - loss: 1.3833 - acc: 0.2650 - val_loss: 1.3869 - val_acc: 0.1928
Epoch 4/10
 - 13s - loss: 1.3823 - acc: 0.2521 - val_loss: 1.3890 - val_acc: 0.1928
Epoch 5/10
 - 13s - loss: 1.3794 - acc: 0.2799 - val_loss: 1.3903 - val_acc: 0.1928
Epoch 6/10
 - 13s - loss: 1.3798 - acc: 0.2650 - val_loss: 1.3903 - val_acc: 0.1928
Epoch 7/10
 - 13s - loss: 1.3812 - acc: 0.2500 - val_loss: 1.3890 - val_acc: 0.1928
Epoch 8/10
 - 13s - loss: 1.3806 - acc: 0.2650 - val_loss: 1.3859 - val_acc: 0.1928
Epoch 9/10
 - 13s - loss: 1.3800 - acc: 0.2863 - val_loss: 1.3843 - val_acc: 0.1928
Epoch 10/10
 - 13s - loss: 1.3821 - acc: 0.2714 - val_loss: 1.3823 - val_acc: 0.1928


In [70]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=50,
verbose=2)

Train on 468 samples, validate on 83 samples
Epoch 1/50
 - 4s - loss: 1.3902 - acc: 0.2692 - val_loss: 1.3625 - val_acc: 0.3855
Epoch 2/50
 - 0s - loss: 1.3529 - acc: 0.3803 - val_loss: 1.1959 - val_acc: 0.6386
Epoch 3/50
 - 0s - loss: 1.1934 - acc: 0.5107 - val_loss: 1.0149 - val_acc: 0.6627
Epoch 4/50
 - 0s - loss: 0.9921 - acc: 0.5897 - val_loss: 0.8983 - val_acc: 0.6867
Epoch 5/50
 - 0s - loss: 0.8927 - acc: 0.6239 - val_loss: 1.0398 - val_acc: 0.6506
Epoch 6/50
 - 0s - loss: 0.8523 - acc: 0.6496 - val_loss: 1.1490 - val_acc: 0.6747
Epoch 7/50
 - 0s - loss: 0.8795 - acc: 0.6132 - val_loss: 0.9142 - val_acc: 0.6506
Epoch 8/50
 - 0s - loss: 0.8809 - acc: 0.6560 - val_loss: 0.9097 - val_acc: 0.6867
Epoch 9/50
 - 0s - loss: 0.7855 - acc: 0.6880 - val_loss: 0.6823 - val_acc: 0.6988
Epoch 10/50
 - 0s - loss: 0.6186 - acc: 0.7564 - val_loss: 0.7288 - val_acc: 0.7229
Epoch 11/50
 - 0s - loss: 0.5827 - acc: 0.7607 - val_loss: 0.6239 - val_acc: 0.7831
Epoch 12/50
 - 0s - loss: 0.5431 - acc: 

In [45]:
y_prob = model.predict(x_test) 
y_classes = y_prob.argmax(axis=-1)
print(len(y_classes))
print(y_test)

83
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0.

In [42]:
y_prob1 = model.predict(x_train) 
y_classes1 = y_prob1.argmax(axis=-1)
print(y_classes1)
print(Y_test)

[0 0 3 0 3 0 1 1 2 2 0 0 0 3 1 3 3 0 3 0 1 0 2 0 0 2 2 3 3 0 1 1 3 1 0 1 2
 0 2 2 2 1 3 1 2 3 1 0 3 2 1 1 3 0 2 0 2 0 2 2 2 0 0 3 3 0 3 0 0 0 3 0 3 0
 2 3 3 2 2 2 0 3 3 1 3 1 0 3 1 2 0 2 3 0 1 3 0 1 3 2 3 0 0 3 0 0 0 1 2 1 0
 2 0 3 1 2 1 0 1 1 1 2 0 0 0 2 0 0 2 0 2 1 2 2 2 2 1 3 2 2 3 1 1 0 2 3 2 1
 2 1 2 0 3 0 2 2 3 0 1 2 1 1 3 2 0 1 2 2 1 3 2 2 3 1 0 0 0 0 2 1 3 3 3 1 3
 3 1 2 3 0 0 2 1 1 1 3 0 3 1 1 3 0 3 3 0 0 2 0 2 1 2 0 1 2 1 1 1 2 1 0 1 3
 2 1 2 3 0 1 2 3 0 2 1 3 1 0 3 1 3 3 0 1 2 3 1 0 2 0 0 1 0 2 1 0 2 0 2 3 0
 1 0 0 0 3 3 0 2 0 0 0 2 3 1 1 0 1 2 2 3 1 0 0 3 0 1 1 2 2 0 0 0 3 3 1 2 2
 1 3 2 3 3 1 0 1 1 1 0 0 1 0 2 2 2 1 0 3 0 0 0 0 2 1 2 1 3 0 1 3 2 2 2 3 1
 2 2 0 0 1 2 0 0 3 1 3 3 3 1 3 1 0 2 2 1 1 1 0 0 3 1 0 2 1 2 3 0 3 0 0 2 2
 0 3 1 2 0 0 2 2 2 0 0 3 1 0 2 0 3 1 0 3 0 1 1 1 2 1 2 0 1 2 0 2 2 1 1 0 2
 3 1 3 1 0 1 2 1 1 0 1 1 2 2 1 2 3 2 3 3 3 1 3 0 0 2 3 2 1 2 2 1 2 0 0 1 2
 2 0 2 0 0 2 2 3 1 3 0 2 2 3 0 0 0 3 2 1 1 1 2 3]
193    2
248    2
277    2
320    3
374    3
175  

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [ ]:
dummy_y.shape

In [ ]:
dummy_y[200]

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,dummy_y,test_size=0.15)

In [ ]:
max_len=int(data['pattern'].str.len().max())
print(max_len)
min_len=int(data['pattern'].str.len().min())
print(min_len)

In [ ]:
max_words = 4
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [ ]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=1,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
class=mode